Создадим простой пример работы с бд

In [ ]:
# Создайте файл debug_db.py
import chromadb
import os

# Подключитесь к вашей базе
client = chromadb.PersistentClient(path="./chroma_db")

# Смотрим коллекции
collections = client.list_collections()
print(f"Коллекции: {[c.name for c in collections]}")

# Смотрим содержимое
for collection in collections:
    print(f"\n=== Коллекция: {collection.name} ===")
    count = collection.count()
    print(f"Записей: {count}")
    
    if count > 0:
        # Смотрим 3 записи
        results = collection.peek(limit=3)
        print(results)
